In [7]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

#, store={}, item_uuids=[]
env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 


set_url = '/search/?award.project=4DN&experiments_in_set.experiment_type=ChIP-seq&experimentset_type=replicate&lab.display_title=Feng+Yue+Lab%2C+PSU&type=ExperimentSetReplicate&status=pre-release&status=released&status=released%20to%20project'

set_to_release = [i['uuid'] for i in ff_utils.search_metadata(set_url, my_auth)]


store, uuids = record_object_es(set_to_release, my_auth, schema_name, add_pc_wfr=True, store_frame='object', ignore_field=['experiment_relation'])
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1))

217
217
6.669574975967407


In [10]:
# TODO
# Check audits

# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = True
delete_problematic = False

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = ['experiment_hi_c', 'experiment_mic', 'experiment_seq']
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(store['experiment_hi_c']), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum') or not a_file.get('md5sum'):
        print(a_file['accession'], 'problems with md5')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)

# check processed files
print('\nFILE PROCESSED CHECK')
for a_file in store['file_processed']:
    if a_file['file_format'] == '/file-formats/pairs/':
        if not a_file.get('quality_metric'):
            print(a_file['accession'], 'missing Pairsqc')
    if not a_file.get('source_experiments'):
        print(a_file['accession'], 'missing source experiments')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)   

# check wfrs
print('\nWFR CHECK')
for wfr in store['workflow_run_awsem']:
    if wfr['run_status'] != 'complete':
        print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i,x['uuid'],x['status']] for x in store[i] if x['status'] in ['deleted', 'replaced', 'archived', 'revoked']]
    if weird:
        for case in weird:
            print(case)
        print()


EXPSET CHECK

EXP CHECK

FILE FASTQ CHECK

FILE PROCESSED CHECK
4DNFIMFNQVZ7 missing source experiments
4DNFI16QXNFN missing source experiments
4DNFI48TBSF2 missing Pairsqc
4DNFI48TBSF2 missing source experiments
4DNFI9XEXNCQ missing source experiments
4DNFIAI5WWUK missing Pairsqc
4DNFIAI5WWUK missing source experiments
4DNFITP54BLW missing source experiments
4DNFIBW0YEBT missing source experiments
4DNFI3BKDEZD missing source experiments
4DNFIWOSOLRX missing source experiments
4DNFIZOHD8A5 missing source experiments
4DNFIIU1S6X9 missing Pairsqc
4DNFIIU1S6X9 missing source experiments
4DNFIF3MKFB4 missing source experiments
4DNFI6KOULN3 missing source experiments
4DNFI0XZF7IM missing source experiments
4DNFIK04WEG0 missing source experiments
4DNFI6S01MM8 missing source experiments
4DNFIMDUVALM missing Pairsqc
4DNFIMDUVALM missing source experiments
4DNFIANXLTA3 missing Pairsqc
4DNFIANXLTA3 missing source experiments
4DNFIRXWQHY8 missing Pairsqc
4DNFIRXWQHY8 missing source experiments
4D

In [11]:
# Check status
change_status = 'released to project'

common_types = ['lab', 'user', 'award','ontology', 'ontology_term', 
                'file_format', 'software', 'workflow', 'static_section',
                'organism', 'vendor', 'file_reference', 'individual_human', 
                'enzyme', 'biosource', 'file_format']

print('COMMON ITEMS')
for a_type in store:
    if a_type in common_types:
        for raw_data in store[a_type]:
            if raw_data['status'] not in ['released', 'current']:
                print('COMMON ITEM NOT RELEASED', a_type,raw_data['status'], raw_data['uuid'])
                # ff_utils.patch_metadata({'status':'released'}, obj_id=raw_data['uuid'] ,key=my_auth)
print('---------------------------------\n')

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] == 'released':
            continue
        if raw_data['status'] != change_status:
            print(a_type, raw_data['uuid'], raw_data['status'])
            # ff_utils.patch_metadata({'status':'released to project'}, obj_id=raw_data['uuid'] ,key=my_auth)
            



COMMON ITEMS
---------------------------------

file_processed fe159aab-e205-4e8e-9fad-7d3280d44597 uploaded
file_processed e3b7aba2-dfda-4d1c-9ee7-5e72d11d595a uploaded
file_processed ca1e0ec5-f4c6-420b-84c9-971ea1630c50 uploaded
file_processed a266151e-8398-4cb8-b941-e50fb3e2e28a uploaded
file_processed 93313684-c325-4ca5-98c3-88bbbf43c1ba uploaded
file_processed 8cc08e0d-0b5a-48b5-b3fb-59d12824d61a uploaded
file_processed 702dcee3-fa90-4adc-b175-bfdedfde5244 uploaded
file_processed 4a98e3d2-3336-40bf-bc10-0375dff0a713 uploaded
file_processed 0f5a7396-22bc-48c4-849a-e51260a3d1c1 uploaded
file_processed f1673fcd-0741-4e4a-8001-878de6514ab7 uploaded
file_processed e4eca90c-d3ad-49e3-82b0-e0cad4ec3f7c uploaded
file_processed e1604745-de6d-447d-ab74-3f20b8dddc60 uploaded
file_processed cdef3e6e-043d-40ad-85a8-87650660423b uploaded
file_processed 951c91e5-405f-4c90-a732-5cf298375885 uploaded
file_processed 86fb8a2d-0ac0-447c-aaf6-23d52c3ec030 uploaded
file_processed 7e3c967f-412f-446b-806

In [4]:
# Release the items
action = True
change_status = 'released'

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print('deleted data', a_type ,raw_data['uuid'])
            continue        
        if raw_data['status'] not in ['current', 'released']:    
            counter +=1
            print(a_type, raw_data['status'], raw_data['uuid'])
            patch_data = {'status': change_status}
            if action:
                if change_status == 'released':
                    if a_type in ['publication', 'user', 'award', 'vendor', 'organism', ]:
                        ff_utils.patch_metadata({'status': 'current'}, obj_id=raw_data['uuid'] ,key=my_auth)
                    else:
                        ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                else:
                    ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                
        else:
            print('ALREADY RELEASED/TO PROJECT', a_type, raw_data['status'], raw_data['uuid'])
            pass
            
print(counter)

workflow_run_awsem in review by lab f6548d67-2e34-4133-ad24-e6f2e40b0ee9
ALREADY RELEASED/TO PROJECT workflow_run_awsem released ee2634cc-9643-4e16-b9a1-a7632012c1e8
ALREADY RELEASED/TO PROJECT workflow_run_awsem released d67c33aa-689e-4c0b-bc5a-85e781b81585
workflow_run_awsem in review by lab d3894c90-bf46-49e3-8c0d-b5646ca80f54
workflow_run_awsem in review by lab a0f1ad9c-f909-4f3a-a2cc-bc0c91ba886e
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 9021d23e-6d25-4c4a-a479-987ce22edac0
workflow_run_awsem in review by lab 8fd0dc77-606a-4ec9-b8c4-b8b7d5ab4fe3
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 8c8daae8-a7a2-40c8-8bb1-8bf06968f704
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 840460a8-f19f-4fe7-a496-bbf7c9e2d095
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 7d0d7df5-f6eb-4f6b-82f7-d518b8f9c4ae
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 5de1f6e9-50a9-40a5-a3ae-518648290e09
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 4